In [1]:
#import to boild model
from tensorflow.keras.models import Model
# lstm dense and input to biuld text encoder and decoder
from tensorflow.keras.layers import Input,LSTM,Dense
# to better work with numpy arrays
import numpy as np
# disable gpu exegution(doing this due to my currentb divice condition)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
#bath size for training
batch_size =64
# number of times to repatedly run the code for better accuracy
epochs=100
# diamonsinality for encoding 
latent_dim=256
#number of samples to train on
num_samples = 10000
# path for data
data_path = "fra.txt"

In [2]:
#stores input sentenses
input_texts = []
#stores output sentenses
target_texts = []
#stores all unique charectors in input texts
input_characters = set()
#stores all unique charectors in target texts
target_characters = set()
#opeans and reads file
with open(data_path, "r", encoding="utf-8") as f:
    #every line is stored in an array called as lines
    lines = f.read().split("\n")

#loops over every line uptto 10k lines
for line in lines[: min(num_samples, len(lines) - 1)]:
    #x,y is splitted on \t(tab)
    input_text, target_text, _ = line.split("\t")
    # We use tab as the start sequence character
    # for the targets and \n as end sequence character.
    target_text = "\t" + target_text + "\n"
    #appending input and output texts to above array
    input_texts.append(input_text)
    target_texts.append(target_text)
    #adding all unique charectors in input text to input texts set
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
        #adding all unique charectors in input text to target texts set
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [3]:
# sorts the lettrs in input charectors
input_characters = sorted(list(input_characters))
# sorts the lettrs in target charectors
target_characters = sorted(list(target_characters))
# size of input charectors
num_encoder_tokens = len(input_characters)
# size of output charectors
num_decoder_tokens = len(target_characters)
# maximum length of a sentence in input texts
max_encoder_seq_length = max([len(txt) for txt in input_texts])
# maximum length of a sentence in target texts
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [4]:
# assin charector as key and a number to identify it for input charectors
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
# assin charector as key and a number to identify it for output charectors
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
#basically tokenizing the texts
# can also use tokeniser for this

In [5]:
# preapring data for 1 heart encoding putting zeroes in following diamentions
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
# we can also simply use to categorical

In [6]:
# 1 heart encoding the data
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [7]:
# design encoder using lstm
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard encoder_outputs and only keep the states
encoder_states = [state_h, state_c]

# Set up the decoder, using encoder_states as initial state
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#compiling the model
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

In [25]:
#training the model
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

Epoch 1/100
125/125 [==============================] - 31s 233ms/step - loss: 0.3675 - accuracy: 0.8893 - val_loss: 0.4686 - val_accuracy: 0.8624
Epoch 2/100
125/125 [==============================] - 29s 230ms/step - loss: 0.3528 - accuracy: 0.8937 - val_loss: 0.4695 - val_accuracy: 0.8621
Epoch 3/100
125/125 [==============================] - 29s 232ms/step - loss: 0.3391 - accuracy: 0.8980 - val_loss: 0.4549 - val_accuracy: 0.8663
Epoch 4/100
125/125 [==============================] - 29s 230ms/step - loss: 0.3261 - accuracy: 0.9017 - val_loss: 0.4463 - val_accuracy: 0.8688
Epoch 5/100
125/125 [==============================] - 29s 231ms/step - loss: 0.3139 - accuracy: 0.9054 - val_loss: 0.4460 - val_accuracy: 0.8692
Epoch 6/100
125/125 [==============================] - 29s 234ms/step - loss: 0.3022 - accuracy: 0.9088 - val_loss: 0.4417 - val_accuracy: 0.8718
Epoch 7/100
125/125 [==============================] - 29s 234ms/step - loss: 0.2912 - accuracy: 0.9121 - val_loss: 0.4437 -

In [26]:
#saving the model
# model.save("best_model.h5")

In [16]:
#loding the model
import tensorflow as tf
model = tf.keras.models.load_model("best_model.h5")

In [28]:
# Define sampling models
# Restore the model and construct the encoder and decoder.

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [29]:
for seq_index in range(20):
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("Input sentence:", input_texts[seq_index+100])
    print("Decoded sentence:", decoded_sentence)
    print("-")

Input sentence: Got it?
Decoded sentence: Va !

-
Input sentence: Hop in.
Decoded sentence: Va !

-
Input sentence: Hop in.
Decoded sentence: Va !

-
Input sentence: Hug me.
Decoded sentence: Va !

-
Input sentence: Hug me.
Decoded sentence: Salut.

-
Input sentence: I fell.
Decoded sentence: Salut.

-
Input sentence: I fell.
Decoded sentence: Fuyez !

-
Input sentence: I fled.
Decoded sentence: Fuyez !

-
Input sentence: I knit.
Decoded sentence: Fuyez !

-
Input sentence: I know.
Decoded sentence: Fuyez !

-
Input sentence: I left.
Decoded sentence: Fuyez !

-
Input sentence: I left.
Decoded sentence: Fuyez !

-
Input sentence: I lied.
Decoded sentence: Fuyez !

-
Input sentence: I lost.
Decoded sentence: Fuyez !

-
Input sentence: I paid.
Decoded sentence: Fuyez !

-
Input sentence: I paid.
Decoded sentence: Fuyez !

-
Input sentence: I paid.
Decoded sentence: Fuyez !

-
Input sentence: I pass.
Decoded sentence: Fuyez !

-
Input sentence: I quit.
Decoded sentence: Fuyez !

-
Input s

In [30]:
from nltk.translate.bleu_score import corpus_bleu
from tqdm import tqdm
actual=[]
predicted=[]
for i in tqdm(range(1000)):
    actual.append(target_texts[i])
    
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    predicted.append(decoded_sentence)

100%|██████████| 1000/1000 [05:38<00:00,  2.95it/s]


In [31]:
#was getting 0.48 score when i ran my model for 100 epochs
print("score1 = ",corpus_bleu(actual,predicted,weights=(1,0,0,0)))
print("score2 = ",corpus_bleu(actual,predicted))

C:\Users\NITHISH\AppData\Local\Programs\Python\Python39\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\NITHISH\AppData\Local\Programs\Python\Python39\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\NITHISH\AppData\Local\Programs\Python\Python39\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of l

score1 =  0.486375
score2 =  1.5214270536628157e-231
